# Setup

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd gdrive/'My Drive'/'Vanderbilt - MSDS'/'Second Year'/vandy_hacks/data

/content/gdrive/My Drive/Vanderbilt - MSDS/Second Year/vandy_hacks/data


In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [44]:
import os
import torch
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader, sampler
from torchvision import models

from pathlib import Path

# Data science tools
import numpy as np
import pandas as pd

# Image manipulations
from PIL import Image
import PIL.Image

# Visualizations
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

## IMPORT MODELS

In [68]:
model_spectogram = torch.load('../model/spectogram.pt', 
                              map_location=torch.device('cpu'))

In [69]:
model_chromagram = torch.load('../model/chromagram.pt', 
                              map_location=torch.device('cpu'))

In [70]:
model_mfcc = torch.load('../model/mfcc.pt', 
                        map_location=torch.device('cpu'))

In [50]:
def transform_image(img_path):
    
    img = PIL.Image.open(img_path, mode='r').convert('RGB')
    
    trans = transforms.Compose([
    transforms.CenterCrop(size=224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])  # Imagenet standards
    ])
    
    return trans(img)

# Get Validation Data

In [71]:
file_names = dict.fromkeys(['normal', 'extra', 'artifact', 'murmur'])

In [72]:
# all have the same file names
for i in ['normal', 'extra', 'artifact', 'murmur']:
  tmp = os.listdir(f'final_valid/spectogram/{i}')
  file_names[i] = tmp

In [74]:
predictions = []
real_class = []

for key, value in file_names.items():
  print(key)
  for img in value:
    tmp_results = []

    # Get Images
    img_spect = transform_image(Path('final_valid', 'spectogram', key, img))
    img_chroma = transform_image(Path('final_valid', 'chromagram', key, img))
    img_mfcc = transform_image(Path('final_valid', 'mfcc', key, img))

    # Make Predictions
    predict_spect = model_spectogram(img_spect.unsqueeze(0)).detach().numpy()
    predict_chroma = model_chromagram(img_chroma.unsqueeze(0)).detach().numpy()
    predict_mfcc = model_mfcc(img_mfcc.unsqueeze(0)).detach().numpy()

    # print(predict_spect)

    # Results
    tmp_results = np.array([predict_spect])
    tmp_results = np.array([predict_spect, 
                            predict_chroma,
                            predict_mfcc])
    
    real_class.append(key)
    predictions.append(tmp_results.mean(axis=0).argmax())


normal
extra
artifact
murmur


In [76]:
df_predictions = pd.DataFrame({
    'prediction': predictions,
    'actual': real_class
})

In [78]:
df_predictions['prediction'].value_counts()

3    102
2      9
0      6
Name: prediction, dtype: int64

In [84]:
df_predictions.loc[df_predictions['prediction'] == 3, 'prediction'] = 'normal'
df_predictions.loc[df_predictions['prediction'] == 2, 'prediction'] = 'murmur'
df_predictions.loc[df_predictions['prediction'] == 0, 'prediction'] = 'artifact'
df_predictions.loc[df_predictions['prediction'] == 1, 'prediction'] = 'extra'

In [86]:
df_predictions['prediction'][0] == df_predictions['actual'][0]

True

In [87]:
df_predictions['results'] = (df_predictions['prediction'] == df_predictions['actual'])

In [89]:
df_predictions['results'].mean()

0.7094017094017094